In [79]:
import glob, os
from bs4 import BeautifulSoup as bs
import re
import collections
import json
import io
import fix,parser

In [80]:
def parse(html,publisher):
    global s,s_body
    if publisher=='sag':
        article=parser.parse_sag(html)
    if publisher=='wly':
        #article=parser.parse_wly(html)
        s = bs(html, 'lxml').find('head')
        s_body=bs(html, 'lxml').find('body')
        source=s.find('meta',{'name':'citation_journal_title'})['content']
        year=s.find('meta',{'name':'citation_publication_date'})['content'][:4]
        vol= s.find('meta',{'name':'citation_volume'})['content']
        iss= s.find('meta',{'name':'citation_issue'})['content']
        title=s.find('meta',{'property':'og:title'})['content']
        creators=s.findAll('meta',{'name':'citation_author'}); names=[c['content'] for c in creators]
        keywords_html=s.findAll('meta',{'name':'citation_keywords'}); keywords=[k['content'] for k in keywords_html]
        url=s.find('meta',{'name':'citation_abstract_html_url'})['content']
        abstract_html=s_body.find('div',{'class':'article-section__content mainAbstract'})

        if abstract_html is not None:
            abstract=abstract_html.find('p').text
        else:
            abstract=''

    article={
        'ex:source'   : source,
        'ex:date'     : year,
        'ex:volume'   : vol,
        'ex:issue'    : iss,
        'ex:title'    : title,
        'ex:creator'  : names,
        'ex:abstract' : abstract,        
        'ex:keyword'  : keywords,
        'ex:url'      : url        
    }
    return article

In [81]:
def src2jsonld(publisher, journal_name,base_folder):
    global articles; articles=[]
    folder=base_folder+"/htmls"
    files=os.listdir(folder)
    l=[]
    i=0
    global e, f
    for f in files:
        i=i+1
        if 'html' in f:
            if i % 10 ==0:
                print i,
            html=open(folder+'/'+f).read()
            article=parse(html,publisher)
            if fix.validate(article)==True:
                articles.append(article)
            else:
                e=e+1
    jsonld_folder='jsonld/'
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    jsonld_file=jsonld_folder+(base_folder+'.jsonld').replace('/','_')
    with io.open(jsonld_file, 'w', encoding="utf-8") as f:
        f.write(jd)
    return (jsonld_file)

In [82]:
with open('journals.json') as data_file:    
    journals = json.load(data_file)

src_folder='src/'
jsonld_folder='jsonld/'

for journal in journals:
    source=journal['jname']
    journal_name=journal['journal']
    data=journal['data']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='src' and publisher=='wly':
            e=0
            print publisher,source,journal_name,
            base_folder=file_type+'/'+publisher+'/'+source
            src2jsonld(publisher,journal_name,base_folder)
            print 'fixed= ',e

wly popmgt Production and Operations Management 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 fixed=  1


In [83]:
article

{'ex:abstract': '',
 'ex:creator': ['Ferguson, Mark', 'Schmidt, Glen', 'Souza, Gilvan'],
 'ex:date': '2010',
 'ex:issue': '2',
 'ex:keyword': [],
 'ex:source': 'Production and Operations Management',
 'ex:title': 'Special Issue of Production and Operations Management: New Product Development, Innovation and Sustainability',
 'ex:url': 'http://onlinelibrary.wiley.com/doi/10.3401/poms.1080.01139/abstract',
 'ex:volume': '19'}